In [4]:
!pip install torch torchvision torchaudio pytorch-lightning scikit-learn

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision.models as models

In [7]:
# Set torchaudio backend (this call is deprecated but needed for .au files)
torchaudio.set_audio_backend("soundfile")

<ipython-input-7-a7bb97d14381>:2: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [8]:
# ----------------------------
# Step 1: Check GPU availability
# ----------------------------
print("Is GPU available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

Is GPU available? False


In [9]:
# ----------------------------
# Step 2: Load dataset from Google Drive
# ----------------------------
# Adjust the path below if your folder structure is different.
dataset_path = '/content/drive/MyDrive/datasets/GTZAN/genres'
file_paths = []
genres = []
for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.au'):
            file_paths.append(os.path.join(root, file))
            genres.append(os.path.basename(root))  # Genre inferred from folder name

df = pd.DataFrame({'file_path': file_paths, 'genre': genres})
print("Dataset loaded. Total files:", len(df))

Dataset loaded. Total files: 1000


In [10]:
# ----------------------------
# Step 3: Label encoding for genres
# ----------------------------
global_le = LabelEncoder()
global_le.fit(df['genre'])
num_classes = len(global_le.classes_)
print("Number of classes:", num_classes)

Number of classes: 10


In [11]:
# ----------------------------
# Step 4: Define the dataset class to compute mel-spectrograms
# ----------------------------
class GTZANDataset(Dataset):
    def __init__(self, df, label_encoder, duration=10, n_mels=128, hop_length=256):
        self.df = df.reset_index(drop=True)
        self.duration = duration            # seconds of audio to load
        self.n_mels = n_mels                # number of mel bins
        self.hop_length = hop_length        # hop length for mel-spectrogram
        self.sample_rate = 22050            # standard sample rate
        self.le = label_encoder             # shared label encoder

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        file_path = row['file_path']
        try:
            waveform, sr = torchaudio.load(file_path)
            # Convert to mono if stereo
            if waveform.shape[0] > 1:
                waveform = torch.mean(waveform, dim=0, keepdim=True)
            target_samples = self.sample_rate * self.duration
            if waveform.shape[1] > target_samples:
                waveform = waveform[:, :target_samples]
            else:
                waveform = F.pad(waveform, (0, target_samples - waveform.shape[1]))
            # Compute mel-spectrogram and convert to decibel scale
            mel_transform = torchaudio.transforms.MelSpectrogram(
                sample_rate=self.sample_rate,
                n_mels=self.n_mels,
                n_fft=2048,
                hop_length=self.hop_length
            )
            mel_spec = mel_transform(waveform)
            mel_spec = torchaudio.transforms.AmplitudeToDB()(mel_spec)
            # Rearrange to shape (time_steps, n_mels)
            mel_spec = mel_spec.squeeze(0).T
            label = torch.tensor(self.le.transform([row['genre']])[0], dtype=torch.long)
            return mel_spec, label
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            default_mel_spec = torch.zeros((self.duration * self.sample_rate // self.hop_length + 1, self.n_mels))
            return default_mel_spec, torch.tensor(0, dtype=torch.long)

In [12]:
# ----------------------------
# Step 5: Split the data into training and validation sets
# ----------------------------
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42, stratify=df['genre'])
train_dataset = GTZANDataset(df_train, label_encoder=global_le, duration=10, n_mels=128, hop_length=256)
val_dataset = GTZANDataset(df_val, label_encoder=global_le, duration=10, n_mels=128, hop_length=256)

In [13]:
# ----------------------------
# Step 6: Create DataLoaders
# ----------------------------
batch_size = 8  # Adjust based on your GPU memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, persistent_workers=True)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [14]:
# ----------------------------
# Step 7: Define the Transfer Learning Model using Pretrained EfficientNet-B3
# ----------------------------
class MusicEfficientNetB3(pl.LightningModule):
    def __init__(self, num_classes, learning_rate=1e-3):
        super(MusicEfficientNetB3, self).__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        # Load pretrained EfficientNet-B3
        self.model = models.efficientnet_b3(pretrained=True)
        # Modify the first conv layer to accept 1 channel instead of 3
        self.model.features[0][0] = nn.Conv2d(
            1,
            self.model.features[0][0].out_channels,
            kernel_size=self.model.features[0][0].kernel_size,
            stride=self.model.features[0][0].stride,
            padding=self.model.features[0][0].padding,
            bias=False
        )
        # Replace the classifier to output the correct number of classes
        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        # x shape: (batch, time_steps, n_mels)
        x = x.unsqueeze(1)  # Add channel dimension: (batch, 1, time_steps, n_mels)
        # Resize to 300x300 to match EfficientNet-B3 input requirements
        x = F.interpolate(x, size=(300, 300), mode="bilinear", align_corners=False)
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        return {"val_loss": loss, "val_acc": acc}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3)
        return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"}}

In [15]:
# ----------------------------
# Step 8: Initialize the model
# ----------------------------
model = MusicEfficientNetB3(num_classes=num_classes, learning_rate=1e-3)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth
100%|██████████| 47.2M/47.2M [00:00<00:00, 128MB/s]


In [16]:
# ----------------------------
# Step 9: Configure the Trainer
# ----------------------------
trainer = Trainer(
    max_epochs=30,  # More epochs for thorough fine-tuning
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    precision="16-mixed",  # Mixed precision for faster training on GPU
    accumulate_grad_batches=4,
    log_every_n_steps=10
)

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:513: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
# ----------------------------
# Step 10: Train and validate the model
# ----------------------------
trainer.fit(model, train_loader, val_loader)
results = trainer.validate(model, dataloaders=val_loader)
print("Validation results:", results)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type         | Params | Mode 
-----------------------------------------------
0 | model | EfficientNet | 10.7 M | train
-----------------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.844    Total estimated model params size (MB)
533       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]